In [51]:
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb

In [54]:
baseball_data = pd.read_excel('C:/workspace/p-tag/KBO_prediction_data/baseball_2023.xlsx')

In [55]:
baseball_data

,순,T_ID,팀,G,AB,PA,RUN,안타,2타,3타,...,희타,희비,OOO,OBP,장타,OPS,wOBA,wRC+,WAR*,WPA
0,1,NC,23,25,81,76,14,26,5,1,...,1,1,0.342,0.363,0.513,0.876,0.389,133.7,0.59,0.21
1,2,LG,23,30,94,81,16,27,6,0,...,1,1,0.333,0.409,0.407,0.816,0.386,118.2,0.53,0.24
2,3,KT,23,31,91,82,20,25,6,1,...,0,2,0.305,0.352,0.512,0.864,0.383,115.5,0.45,0.18
3,4,두산,23,24,83,70,12,18,2,0,...,2,1,0.257,0.346,0.371,0.717,0.331,97.7,0.29,0.64
4,5,키움,23,24,85,78,10,24,7,1,...,0,0,0.308,0.365,0.423,0.788,0.364,107.0,0.23,0.90
5,6,삼성,23,25,74,63,8,17,2,0,...,1,1,0.270,0.356,0.349,0.705,0.334,96.6,0.20,0.22
6,7,KIA,23,24,80,66,10,19,2,0,...,1,1,0.288,0.392,0.364,0.756,0.362,104.7,0.17,0.03
7,8,롯데,23,24,92,78,12,19,3,1,...,2,0,0.244,0.344,0.346,0.691,0.321,85.7,0.16,0.64
8,9,SSG,23,24,77,63,9,13,1,0,...,1,0,0.206,0.342,0.365,0.707,0.332,81.0,0.15,0.03
9,10,한화,23,23,81,78,8,20,5,1,...,0,0,0.256,0.284,0.346,0.630,0.294,64.7,0.00,-0.10


### 팀 명 라벨 인코딩

In [56]:
team_name = ['한화', 'KIA', 'KT', 'LG', '롯데', 'NC', '두산', 'SSG', '삼성', '키움']

for index, name in enumerate(team_name):
    baseball_data['T_ID'] = baseball_data['T_ID'].replace(name,index)

In [57]:
baseball_data

,순,T_ID,팀,G,AB,PA,RUN,안타,2타,3타,...,희타,희비,OOO,OBP,장타,OPS,wOBA,wRC+,WAR*,WPA
0,1,5,23,25,81,76,14,26,5,1,...,1,1,0.342,0.363,0.513,0.876,0.389,133.7,0.59,0.21
1,2,3,23,30,94,81,16,27,6,0,...,1,1,0.333,0.409,0.407,0.816,0.386,118.2,0.53,0.24
2,3,2,23,31,91,82,20,25,6,1,...,0,2,0.305,0.352,0.512,0.864,0.383,115.5,0.45,0.18
3,4,6,23,24,83,70,12,18,2,0,...,2,1,0.257,0.346,0.371,0.717,0.331,97.7,0.29,0.64
4,5,9,23,24,85,78,10,24,7,1,...,0,0,0.308,0.365,0.423,0.788,0.364,107.0,0.23,0.90
5,6,8,23,25,74,63,8,17,2,0,...,1,1,0.270,0.356,0.349,0.705,0.334,96.6,0.20,0.22
6,7,1,23,24,80,66,10,19,2,0,...,1,1,0.288,0.392,0.364,0.756,0.362,104.7,0.17,0.03
7,8,4,23,24,92,78,12,19,3,1,...,2,0,0.244,0.344,0.346,0.691,0.321,85.7,0.16,0.64
8,9,7,23,24,77,63,9,13,1,0,...,1,0,0.206,0.342,0.365,0.707,0.332,81.0,0.15,0.03
9,10,0,23,23,81,78,8,20,5,1,...,0,0,0.256,0.284,0.346,0.630,0.294,64.7,0.00,-0.10


In [58]:
train_data = baseball_data[['RUN', 'AB', 'PA', 'OOO','T_ID','OBP']]

In [59]:
train_data.columns

Index(['RUN', 'AB', 'PA', 'OOO', 'T_ID', 'OBP'], dtype='object')

In [60]:
train_data[['RUN', 'AB', 'PA', 'OOO','T_ID','OBP']] = train_data[['RUN', 'AB', 'PA', 'OOO','T_ID','OBP']]/2
train_data

,RUN,AB,PA,OOO,T_ID,OBP
0,7.0,40.5,38.0,0.1710,2.5,0.1815
1,8.0,47.0,40.5,0.1665,1.5,0.2045
2,10.0,45.5,41.0,0.1525,1.0,0.1760
3,6.0,41.5,35.0,0.1285,3.0,0.1730
4,5.0,42.5,39.0,0.1540,4.5,0.1825
5,4.0,37.0,31.5,0.1350,4.0,0.1780
6,5.0,40.0,33.0,0.1440,0.5,0.1960
7,6.0,46.0,39.0,0.1220,2.0,0.1720
8,4.5,38.5,31.5,0.1030,3.5,0.1710
9,4.0,40.5,39.0,0.1280,0.0,0.1420


In [78]:
import pickle
pickled_RandomForest_model = pickle.load(open('C:/workspace/p-tag/model/random_forest(except vs_t_id).pkl', 'rb'))
pred = pickled_RandomForest_model.predict(train_data)
pred_probs_RandomForest = pickled_RandomForest_model.predict_proba(train_data)
pred_probs_RandomForest


array([[0.52793484, 0.47206516],
       [0.52255203, 0.47744797],
       [0.50286114, 0.49713886],
       [0.58214057, 0.41785943],
       [0.71763133, 0.28236867],
       [0.70387187, 0.29612813],
       [0.61031665, 0.38968335],
       [0.62729101, 0.37270899],
       [0.69974941, 0.30025059],
       [0.77583545, 0.22416455]])

In [79]:
pickled_XGBoost_model = pickle.load(open('C:/workspace/p-tag/model/xgboost(except vs_t_id).pkl', 'rb'))
dtest = xgb.DMatrix(data=train_data)

percent = pickled_XGBoost_model.predict(dtest)
preds = [ 1 if x > 0.5 else 0 for x in percent ]
pred_probs_xgboost = np.array([arr for arr in zip(1-percent,percent)])
pred_probs_xgboost

array([[0.3561728 , 0.6438272 ],
       [0.60276437, 0.39723566],
       [0.42694432, 0.5730557 ],
       [0.30598307, 0.69401693],
       [0.51384974, 0.48615023],
       [0.6291021 , 0.37089786],
       [0.7697217 , 0.23027831],
       [0.4403692 , 0.5596308 ],
       [0.561748  , 0.43825197],
       [0.51655984, 0.4834402 ]], dtype=float32)

In [81]:
pickled_GBM_model = pickle.load(open('C:/workspace/p-tag/model/GBM(except vs_t_id).pkl', 'rb'))

pred = pickled_GBM_model.predict(train_data)
pred_probs_GBM = pickled_GBM_model.predict_proba(train_data)
pred_probs_GBM


array([[0.86244274, 0.13755726],
       [0.92495917, 0.07504083],
       [0.87046124, 0.12953876],
       [0.89471545, 0.10528455],
       [0.95883088, 0.04116912],
       [0.90668055, 0.09331945],
       [0.93454596, 0.06545404],
       [0.97460898, 0.02539102],
       [0.94746199, 0.05253801],
       [0.97825404, 0.02174596]])

In [66]:
pred_probs_xgboost = np.array([arr for arr in zip(1-percent,percent)])
pred_probs_xgboost

array([[0.3561728 , 0.6438272 ],
       [0.60276437, 0.39723566],
       [0.42694432, 0.5730557 ],
       [0.30598307, 0.69401693],
       [0.51384974, 0.48615023],
       [0.6291021 , 0.37089786],
       [0.7697217 , 0.23027831],
       [0.4403692 , 0.5596308 ],
       [0.561748  , 0.43825197],
       [0.51655984, 0.4834402 ]], dtype=float32)

In [70]:
pickled_GBM_model = pickle.load(open('C:/workspace/p-tag/model/GBM(except vs_t_id).pkl', 'rb'))

pred = pickled_GBM_model.predict(train_data)
pred

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [82]:
pickled_LightGBM_model = pickle.load(open('C:/workspace/p-tag/model/LightGBM(except vs_t_id).pkl', 'rb'))
pred = pickled_LightGBM_model.predict(train_data)

pred_probs_LightGBM = pickled_LightGBM_model.predict_proba(train_data)
pred_probs_LightGBM

array([[0.52722522, 0.47277478],
       [0.62272756, 0.37727244],
       [0.49325459, 0.50674541],
       [0.58397604, 0.41602396],
       [0.75823034, 0.24176966],
       [0.80962301, 0.19037699],
       [0.7419121 , 0.2580879 ],
       [0.72893893, 0.27106107],
       [0.80962301, 0.19037699],
       [0.89269523, 0.10730477]])

In [83]:
total_pred_probs = (pred_probs_RandomForest + pred_probs_xgboost + pred_probs_GBM + pred_probs_LightGBM)/4

preds = [1 if total_pred_probs[index][0] < total_pred_probs[index][1] else 0 for index in range(len(total_pred_probs))]
preds

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [84]:
total_pred_probs

array([[0.5684439 , 0.4315561 ],
       [0.66825078, 0.33174922],
       [0.57338032, 0.42661968],
       [0.59170378, 0.40829622],
       [0.73713557, 0.26286442],
       [0.76231939, 0.23768061],
       [0.7641241 , 0.2358759 ],
       [0.69280203, 0.30719797],
       [0.75464561, 0.24535439],
       [0.79083614, 0.20916387]])